In [1]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
## Help Python find our packages
import sys
sys.path.append('..')

import json
import numpy as np
import matplotlib.pyplot as plt
import domain_scoring.domain_scoring as domain_scoring

# Randomness
import random as rn
import tensorflow as tf
import os

In [3]:
path = '../rated_datasets/Rotten Tomato_Potato_1519142479.127663.json'

In [4]:
merlin = json.load(open(path, "r", encoding="utf8"))

In [5]:
path = '../rated_datasets/Rotten Tomato_Potato_1519142479.127663.json'

In [6]:
potato = json.load(open(path, "r", encoding="utf8"))

In [7]:
type_selection = merlin["edge_type_selection"] + merlin["node_type_selection"]
types = []
for pair in type_selection:
    types.append(pair[0])
types

['ACTED_IN', 'PRODUCED', 'DIRECTED', 'WROTE', 'Person', 'Movie']

In [8]:
def extract_mps(data):
    count = 0
    first = True
    batches = []
    batch = []
    for probably_path in data["meta_paths"]:
        if count % 6 == 0:
            # Don't add empty batches
            if len(batch) > 0:
                batches.append(batch)
                batch = []
        else:
            if 'time_to_rate' not in probably_path.keys():
                batch.append(probably_path)
        count += 1
    # append last batch
    if len(batch) > 0:
        batches.append(batch)
    print('#meta-paths:', count - len(batches) - 1)
    return batches

In [9]:
def construct_graph(batches):
    ## Construct rating graph
    from util.datastructures import MetaPathRatingGraph, MetaPath
    graph = MetaPathRatingGraph()

    for batch in batches:
        #ordered = sorted(batch, key=lambda x: float(x['rating']))
        for metapath in batch:
            for another_metapath in batch:
                if metapath is another_metapath:
                    continue
                if float(metapath['rating']) <= float(another_metapath['rating']):
                    graph.add_user_rating(MetaPath.from_list(another_metapath['metapath']), MetaPath.from_list(metapath['metapath']), 
                                          distance=float(another_metapath['rating']) - float(metapath['rating']))
    return graph

In [10]:
## Clean up data // remove time_to_rate from array of ratings.
def clean_up(data):
    batches = extract_mps(data)
    return batches, construct_graph(batches)

In [11]:
p_batches, p_graph = clean_up(potato)
m_batches, m_graph = clean_up(merlin)

#meta-paths: 51
#meta-paths: 51


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [13]:
domain_score = domain_scoring.DomainScoring()

In [14]:
# Preprocess meta-paths
domain_score.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), token_pattern='\\b\\w+\\b')

In [15]:
domain_score.fit(m_graph)

In [16]:
x, y = domain_score._extract_data_labels(m_graph)
x_raw = x
x = domain_score._preprocess(x)

In [17]:
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

In [18]:
labeler = preprocessing.LabelEncoder()
labeler.fit(types)

LabelEncoder()

In [19]:
def to_sequence(in_list):
    return in_list.reshape((-1, 1))

In [20]:
def one_hot(in_sequence, distinct_values):
    sequence = np.zeros((len(in_sequence), distinct_values))
    i = 0
    for point in in_sequence:      
        sequence[i][point] = 1
        i += 1
    return sequence

In [21]:
def preprocess_raw(raw, distinct_values, max_len=9):
    data = []
    for a,b in raw:
        # encode labels to integers (0 is reserved for padding)
        a = np.array(labeler.transform(a.as_list())) + 1
        b = np.array(labeler.transform(b.as_list())) + 1
        # pad to same length
        a, b = pad_sequences([a, b], max_len, padding='post', value=0)
        # merge a and b
        sequence = np.append(a, b)
        # to sequence
        sequence = to_sequence(sequence)
        # one-hot encode because we don't have distances/embeddings
        sequence = one_hot(sequence, distinct_values + 1)
        data.append(sequence)
    return data

In [61]:
x_preprocess = preprocess_raw(x_raw, len(types))

In [62]:
test_size = 0.3
random_state = 42

In [63]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    shuffle=True)

In [64]:
len(x_train)

131

In [65]:
np.array(x_train).shape

(131, 44)

In [66]:
os.environ['PYTHONHASHSEED'] = '0'
def reset_seed():
    np.random.seed(random_state)
    rn.seed(random_state)
    tf.set_random_seed(random_state)

In [82]:
callbacks = [
    EarlyStopping(monitor='loss', patience=2)
]

In [83]:
# Reproducible results
reset_seed()
# Build model
model = Sequential()
model.add(Dense(44, activation='relu', input_dim=44))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
model.fit(np.array(x_train), np.array(y_train), epochs=25, batch_size=32, callbacks=callbacks)

Epoch 1/25
131/131 [==============================] - 1s 5ms/step - loss: 0.6881 - acc: 0.5649
Epoch 2/25
131/131 [==============================] - 0s 71us/step - loss: 0.6791 - acc: 0.5802
Epoch 3/25
131/131 [==============================] - 0s 99us/step - loss: 0.6766 - acc: 0.6183
Epoch 4/25
131/131 [==============================] - 0s 162us/step - loss: 0.6733 - acc: 0.5954
Epoch 5/25
131/131 [==============================] - 0s 146us/step - loss: 0.6723 - acc: 0.6031
Epoch 6/25
131/131 [==============================] - 0s 136us/step - loss: 0.6678 - acc: 0.6336
Epoch 7/25
131/131 [==============================] - 0s 80us/step - loss: 0.6653 - acc: 0.6641
Epoch 8/25
131/131 [==============================] - 0s 154us/step - loss: 0.6637 - acc: 0.6336
Epoch 9/25
131/131 [==============================] - 0s 154us/step - loss: 0.6607 - acc: 0.6565
Epoch 10/25
131/131 [==============================] - 0s 84us/step - loss: 0.6601 - acc: 0.6412
Epoch 11/25
131/131 [==============

In [84]:
model.evaluate(np.array(x_test), np.array(y_test), batch_size=16)

57/57 [==============================] - 0s 7ms/step


[0.6689000746660065, 0.6491228101546305]

In [70]:
x_p_train, x_p_test, y_p_train, y_p_test = train_test_split(x_preprocess, y,
                                                    test_size=test_size,
                                                    random_state=random_state,
                                                    shuffle=True)

In [71]:
np.array(x_p_train).shape

(131, 18, 7)

In [72]:
# Reproducible results
reset_seed()
# Use sequence classification (RNN/LSTM)
model_rnn = Sequential()
model_rnn.add(SimpleRNN(128, input_shape=(18, 7)))
model_rnn.add(Dropout(0.5))
model_rnn.add(Dense(1, activation='sigmoid'))

model_rnn.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model_rnn.fit(np.array(x_p_train), np.array(y_p_train), batch_size=35, epochs=25, callbacks=[])

Epoch 1/25
131/131 [==============================] - 1s 5ms/step - loss: 0.8841 - acc: 0.5115
Epoch 2/25
131/131 [==============================] - 0s 181us/step - loss: 0.5931 - acc: 0.6641
Epoch 3/25
131/131 [==============================] - 0s 290us/step - loss: 0.5322 - acc: 0.7023
Epoch 4/25
131/131 [==============================] - 0s 356us/step - loss: 0.5153 - acc: 0.7328
Epoch 5/25
131/131 [==============================] - 0s 408us/step - loss: 0.5106 - acc: 0.7252
Epoch 6/25
131/131 [==============================] - 0s 344us/step - loss: 0.5026 - acc: 0.7481
Epoch 7/25
131/131 [==============================] - 0s 248us/step - loss: 0.4430 - acc: 0.7710
Epoch 8/25
131/131 [==============================] - 0s 380us/step - loss: 0.4903 - acc: 0.7786
Epoch 9/25
131/131 [==============================] - 0s 333us/step - loss: 0.4717 - acc: 0.7710
Epoch 10/25
131/131 [==============================] - 0s 336us/step - loss: 0.4846 - acc: 0.7481
Epoch 11/25
131/131 [==========

In [73]:
model_rnn.evaluate(np.array(x_p_test), np.array(y_p_test), batch_size=35)

57/57 [==============================] - 0s 5ms/step


[0.5605788722372892, 0.7719298402468363]